In [1]:
# instalacion de dependencias previas
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

fatal: destination path 'UD_Spanish-AnCora' already exists and is not an empty directory.


# Carga del modelo HMM previamente entrenado

tener un modelo markoviano latente previamente entrenado
significa tener calculadas las probabilidades de transicion y de emision

In [2]:

# cargamos las probabilidades del modelo HMM
import numpy as np

transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
emissionProbdict = np.load('emissionHMM.npy', allow_pickle='TRUE').item()

In [3]:
transitionProbdict

{'NOUN|DET': 0.7050851672261594,
 'PUNCT|NOUN': 0.2173822426010838,
 'ADP|PUNCT': 0.10090433127082342,
 'ADJ|ADP': 0.012436935351402088,
 'NOUN|ADJ': 0.21575826037842416,
 'ADP|NOUN': 0.3234681117132138,
 'SCONJ|ADP': 0.017834095975595446,
 'VERB|SCONJ': 0.08362676056338028,
 'VERB|VERB': 0.029333333333333333,
 'NUM|VERB': 0.019333333333333334,
 'NOUN|NUM': 0.5508571428571428,
 '_|NOUN': 0.05960817007086286,
 'ADP|_': 0.8613312202852615,
 'DET|ADP': 0.5137862255074505,
 'VERB|ADP': 0.05678751613281708,
 '_|VERB': 0.035777777777777776,
 'DET|NOUN': 0.014068361817423927,
 'CCONJ|NOUN': 0.049604001667361404,
 'ADJ|CCONJ': 0.08459422283356259,
 'ADP|ADJ': 0.24936458627506355,
 'NOUN|ADP': 0.19464977120732135,
 'ADJ|NOUN': 0.17038349312213422,
 'PUNCT|ADJ': 0.24795255577520475,
 'VERB|PUNCT': 0.0679041726162145,
 'ADP|VERB': 0.24733333333333332,
 'PRON|DET': 0.017903767251025736,
 'DET|PRON': 0.028384279475982533,
 'PROPN|DET': 0.12109909237846575,
 'PROPN|PROPN': 0.22875494071146246,
 'ADP

In [4]:
emissionProbdict

{'el|DET': 0.3435285341290563,
 'gobernante|NOUN': 0.00020842017507294707,
 ',|PUNCT': 0.45137236236712674,
 'con|ADP': 0.051977003402557787,
 'ganada|ADJ': 0.0002824060999717594,
 'fama|NOUN': 0.00010421008753647353,
 'desde|ADP': 0.008799718409010912,
 'que|SCONJ': 0.6382042253521126,
 'llegó|VERB': 0.0022222222222222222,
 'hace|VERB': 0.009111111111111111,
 '16|NUM': 0.011428571428571429,
 'meses|NOUN': 0.0028136723634847854,
 'al|_': 0.2765451664025357,
 'a|ADP': 0.15205913410770855,
 'poder|NOUN': 0.0011463109629012088,
 'de|ADP': 0.4614572333685322,
 'explotar|VERB': 0.00044444444444444447,
 'máximo|NOUN': 0.00020842017507294707,
 'su|DET': 0.04351610095735422,
 'oratoria|NOUN': 0.00010421008753647353,
 'y|CCONJ': 0.7771664374140302,
 'acusado|ADJ': 0.0008472182999152782,
 'por|ADP': 0.05972075560248739,
 'sus|DET': 0.017282108665920677,
 'detractores|NOUN': 0.0003126302626094206,
 'incontinencia|NOUN': 0.00010421008753647353,
 'verbal|ADJ': 0.0005648121999435188,
 'enmudeció|VER

In [5]:
# identificamos las categorias gramaticales 'upos' unicas en el corpus
stateSet = set([w.split('|')[1] for w in list(emissionProbdict.keys())])
stateSet

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 '_'}

In [6]:
# enumeramos las categorias con numeros para asignar a
# las columnas de la matriz de Viterbi

tagStateDict = {}

for i, state in enumerate(stateSet):
    tagStateDict[state] = i
tagStateDict

{'VERB': 0,
 'CCONJ': 1,
 'AUX': 2,
 'NUM': 3,
 '_': 4,
 'PRON': 5,
 'ADV': 6,
 'INTJ': 7,
 'ADJ': 8,
 'ADP': 9,
 'SYM': 10,
 'PUNCT': 11,
 'DET': 12,
 'SCONJ': 13,
 'PART': 14,
 'NOUN': 15,
 'PROPN': 16}

# Distribucion inicial de estados latentes

$$
\rho_i^{(0)}
$$

In [7]:
# Calculamos distribución inicial de estados

#@title leyendo el corpus AnCora
from conllu import parse_incr

wordList = []
data_file = open(
    "./UD_Spanish-AnCora/es_ancora-ud-dev.conllu",
    "r",
    encoding="utf-8",
)

count = 0 # cuento la longitud del corpus
initTagStateProp = {}  # \rho_i^{(0)} probabilidad de encontrar cierta categoria gramatical al inicio de una frase

for tokenlist in parse_incr(data_file):
    count +=1
    tag = tokenlist[0]['upos']
    if tag in initTagStateProp.keys():
        initTagStateProp[tag] += 1
    else:
        initTagStateProp[tag] = 1

for key in initTagStateProp.keys():
    initTagStateProp[key] /= count

initTagStateProp


{'DET': 0.36275695284159615,
 'PROPN': 0.1124546553808948,
 'ADP': 0.15538089480048367,
 'PRON': 0.06348246674727932,
 'SCONJ': 0.02418379685610641,
 'ADV': 0.056831922611850064,
 'PUNCT': 0.08222490931076179,
 'VERB': 0.021160822249093107,
 'ADJ': 0.010882708585247884,
 'CCONJ': 0.032648125755743655,
 'NOUN': 0.02720677146311971,
 '_': 0.009068923821039904,
 'INTJ': 0.0006045949214026602,
 'AUX': 0.019347037484885126,
 'NUM': 0.01995163240628779,
 'PART': 0.0018137847642079807}

In [8]:
# verificamos que la suma de las probabilidades es 1 (100%)
mylist = np.array([initTagStateProp[k] for k in initTagStateProp.keys()])
mylist.sum()

1.0

# Construcción del algoritmo de Viterbi






Dada una secuencia de palabras $\{p_1, p_2, \dots, p_n \}$, y un conjunto de categorias gramaticales dadas por la convención `upos`, se considera la matriz de probabilidades de Viterbi así:

$$
\begin{array}{c c}
\begin{array}{c c c c}
\text{ADJ} \\
\text{ADV}\\
\text{PRON} \\
\vdots \\
{}
\end{array} 
&
\left[
\begin{array}{c c c c}
\nu_1(\text{ADJ}) & \nu_2(\text{ADJ}) & \dots  & \nu_n(\text{ADJ})\\
\nu_1(\text{ADV}) & \nu_2(\text{ADV}) & \dots  & \nu_n(\text{ADV})\\ 
\nu_1(\text{PRON}) & \nu_2(\text{PRON}) & \dots  & \nu_n(\text{PRON})\\
\vdots & \vdots & \dots & \vdots \\ \hdashline
p_1 & p_2 & \dots & p_n 
\end{array}
\right] 
\end{array}
$$

Donde las probabilidades de la primera columna (para una categoria $i$) están dadas por: 

$$
\nu_1(i) = \underbrace{\rho_i^{(0)}}_{\text{probabilidad inicial}} \times \underbrace{P(p_1 \vert i)}_{\text{emisión}}
$$

luego, para la segunda columna (dada una categoria $j$) serán: 

$$
\nu_2(j) = \max_i \{ \nu_1(i) \times \underbrace{P(j \vert i)}_{\text{transición}} \times \underbrace{P(p_2 \vert j)}_{\text{emisión}} \}
$$

así, en general las probabilidades para la columna $t$ estarán dadas por: 

$$
\nu_{t}(j) = \max_i \{ \overbrace{\nu_{t-1}(i)}^{\text{estado anterior}} \times \underbrace{P(j \vert i)}_{\text{transición}} \times \underbrace{P(p_t \vert j)}_{\text{emisión}} \}
$$

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def ViterbiMatrix(secuencia, transitionProbdict=transitionProbdict, emissionProbdict=emissionProbdict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = word_tokenize(secuencia)
  viterbiProb = np.zeros((17, len(seq)))  # upos tiene 17 categorias

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbdict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag]

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys():
        # miramos estados de la col anterior
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        viterbiProb[tag_row, col] = max(possible_probs)
  
  return viterbiProb

matrix = ViterbiMatrix('el mundo es pequeño')
matrix

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.47448483e-04, 2.91245828e-10, 0.00000000e+00],
       [0.00000000e+00, 2.00411724e-05, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.01922433e-10],
       [0.00000000e+00, 0.00000000e+00, 5.02871314e-09, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.76142797e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e

In [ ]:
def ViterbiTags(secuencia, transitionProbdict=transitionProbdict, emissionProbdict=emissionProbdict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = word_tokenize(secuencia)
  viterbiProb = np.zeros((17, len(seq)))  # upos tiene 17 categorias

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbdict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag]

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys():
        # miramos estados de la col anterior
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        viterbiProb[tag_row, col] = max(possible_probs)

    # contruccion de secuencia de tags
    res = []
    for i, p in enumerate(seq):
      for tag in tagStateDict.keys():
        if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
          res.append((p, tag))
      
  return res

ViterbiTags('el mundo es muy pequeño')

[('el', 'DET'),
 ('mundo', 'NOUN'),
 ('es', 'AUX'),
 ('muy', 'ADV'),
 ('pequeño', 'ADJ')]

In [ ]:
ViterbiTags('estos instrumentos han de rasgar')

[('estos', 'DET'),
 ('instrumentos', 'NOUN'),
 ('han', 'AUX'),
 ('de', 'ADP'),
 ('rasgar', 'VERB')]

# Entrenamiento directo de HMM con NLTK

* clase en python (NLTK) de HMM: https://www.nltk.org/_modules/nltk/tag/hmm.html

In [ ]:
#@title ejemplo con el Corpus Treebank en ingles
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
train_data = treebank.tagged_sents()[:3900]

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [ ]:
#@title estructura de la data de entrenamiento
train_data

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [ ]:
#@title HMM pre-construido en NLTK
from nltk.tag import hmm
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(train_data)
tagger

<HiddenMarkovModelTagger 46 states and 12385 output symbols>

In [ ]:
tagger.tag("Pierre Vinken will get old".split())

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 ('will', 'MD'),
 ('get', 'VB'),
 ('old', 'JJ')]

In [ ]:
#@title training accuracy
tagger.evaluate(treebank.tagged_sents()[:3900])

0.9815403947224078

## Ejercicio de práctica

**Objetivo:** Entrena un HMM usando la clase `hmm.HiddenMarkovModelTrainer()` sobre el dataset `UD_Spanish_AnCora`.

1. **Pre-procesamiento:** En el ejemplo anterior usamos el dataset en ingles `treebank`, el cual viene con una estructura diferente a la de `AnCora`, en esta parte escribe código para transformar la estructura de `AnCora` de manera que quede igual al `treebank` que usamos así:

$$\left[ \left[ (\text{'El'}, \text{'DET'}), (\dots), \dots\right], \left[\dots \right] \right]$$

In [ ]:
# desarrolla tu código aquí 


2. **Entrenamiento:** Una vez que el dataset esta con la estructura correcta, utiliza la clase `hmm.HiddenMarkovModelTrainer()` para entrenar con el $80 \%$ del dataset como conjunto de `entrenamiento` y $20 \%$ para el conjunto de `test`.

**Ayuda:** Para la separacion entre conjuntos de entrenamiento y test, puedes usar la funcion de Scikit Learn: 

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

En este punto el curso de Machine Learning con Scikit Learn es un buen complemento para entender mejor las funcionalidades de Scikit Learn: https://platzi.com/cursos/scikitlearn-ml/ 

In [ ]:
# desarrolla tu código aquí


3. **Validación del modelo:** Un vez entrenado el `tagger`, calcula el rendimiento del modelo (usando `tagger.evaluate()`) para los conjuntos de `entrenamiento` y `test`.



In [ ]:
#desarrolla tu código aquí
